In [1]:
import torch

import pyro
import pyro.poutine as poutine
import pyro.distributions as dist

ability_dim = 1
num_person = 10000
num_item = 100

device = torch.device('cpu')

In [2]:
"""
Docstring:
Creates a normal (also called Gaussian) distribution parameterized by loc and scale.

Args:
loc (float or Tensor): mean of the distribution (often referred to as mu)
scale (float or Tensor): standard deviation of the distribution (often referred to as sigma)
"""
annealing_factor = 1
nonlinear = False
mask = None
response = None

# loc : 평균, scale : standard deviation?
ability_prior = dist.Normal(torch.zeros(num_person, ability_dim),
                            torch.ones(num_person, ability_dim))

with poutine.scale(scale=annealing_factor):
    ability = pyro.sample("ability", ability_prior)
        
item_feat_prior = dist.Normal(
    torch.zeros((num_item, 1), device=device), 
    torch.ones((num_item, 1), device=device),
)

item_feat = pyro.sample("item_feat", item_feat_prior)
difficulty = item_feat

logit = (torch.sum(ability, dim=1, keepdim=True) + difficulty.T).unsqueeze(2)

response_mu = torch.sigmoid(logit)

response_dist = dist.Bernoulli(response_mu)
response = pyro.sample("response", response_dist)

In [9]:
import os

OUT_DIR = 'data'

dataset = {'response': response, 'ability': ability, 'item_feat': item_feat}
torch.save(dataset, os.path.join(OUT_DIR, '10000_100_simulation.path'))